# Gradient Boosting Regression Model (Version 2)

Regression counterpart of Gradient Boosted Tree, supported by [`sklearn.ensemble.GradientBoostingRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) function of ScikitLearn

### Summary

| Techniques                     | Used / Description           |
| ------------------------------ | ---------------------------- |
| Handling Unknown Variables     | Drop Rows                    |
| Handling Categorical Variables | Drop Columns (Drop Features) |
| Handling Class Imbalance       | Not Applied                  |
| Handling Outliers              | Not Applied                  |

### Results

| Metric                 | Value   |
| ---------------------- | ------- |
| RMSE (Lower is better) | 0.7816  |
| R2 (Higher is better)  | 0.5294  |


### Preprocessing Stage

In [10]:
import numpy as np
import pandas as pd
import random
import optuna

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler


In [3]:
X_train = pd.read_csv('../../cleaned-data/X_train.csv')
y_train = pd.read_csv('../../cleaned-data/y_train.csv')

X_test = pd.read_csv('../../cleaned-data/X_test.csv')
y_test = pd.read_csv('../../cleaned-data/y_test.csv')

In [4]:
X_train.head()

,latitude,longitude,land_use_label,distance_to_waterbody,distance_to_open_space,subzone,planning_area,region,elevation,temp_2024_04_07_min,...,built-up,bare / sparse vegetation,snow and ice,permanent water bodies,herbaceous wetland,mangroves,moss and lichen,min_ndvi,mean_ndvi,max_ndvi
0,1.327345,103.776261,ROAD,0.005491,0.000305,HOLLAND ROAD,BUKIT TIMAH,CENTRAL REGION,34,28.880736,...,128,1,0,1,0,0,0,0.1176063463,0.2107233339,0.3355351585
1,1.362310,103.885041,RESIDENTIAL,0.002163,0.002288,KOVAN,HOUGANG,NORTH-EAST REGION,14,33.603571,...,183,1,0,0,0,0,0,0.06873453002,0.1237388913,0.1772913102
2,1.304792,103.740678,BUSINESS 2,0.001660,0.001437,PENJURU CRESCENT,JURONG EAST,WEST REGION,10,28.880736,...,251,8,0,33,0,0,0,0.03399855502,0.07334574643,0.1149060753
3,1.432131,103.793028,ROAD,0.002688,0.002472,WOODLANDS SOUTH,WOODLANDS,NORTH REGION,32,30.168782,...,-,-,-,-,-,-,-,-,-,-
4,1.303530,103.820861,CIVIC & COMMUNITY INSTITUTION,0.011124,0.004127,RIDOUT,TANGLIN,CENTRAL REGION,17,30.168782,...,63,1,0,0,0,0,0,0.09017470784,0.2076336658,0.3255961435


In [5]:
# Combine X and y to make sure that the oversampling is done correctly
X_train = pd.concat([X_train, y_train], axis=1)
X_test = pd.concat([X_test, y_test], axis=1)

- Drop subzone and planning area columns
- Replace land use label by one hot encoding
- Drop temperature data, since they are not independent variables

In [6]:
X_train.columns

Index(['latitude', 'longitude', 'land_use_label', 'distance_to_waterbody',
       'distance_to_open_space', 'subzone', 'planning_area', 'region',
       'elevation', 'temp_2024_04_07_min', 'temp_2024_04_07_max',
       'temp_2024_04_07_median', 'temp_2024_04_08_min', 'temp_2024_04_08_max',
       'temp_2024_04_08_median', 'temp_2024_04_09_min', 'temp_2024_04_09_max',
       'temp_2024_04_09_median', 'temp_2024_04_10_min', 'temp_2024_04_10_max',
       'temp_2024_04_10_median', 'Total_x', 'HDB Total',
       'Condominiums & Other Apartments', 'Landed Properties_x',
       'Other Dwellings_x', 'Floor_below_60', 'Floor_60-80', 'Floor_80-100',
       'Floor_100-120', 'Floor_above_120', 'Below $1,000', '$1,000 - $1,999',
       '$2,000 - $2,999', '$3,000 - $3,999', '$4,000 - $4,999',
       '$5,000 - $5,999', '$6,000 - $6,999', '$7,000 - $7,999',
       '$8,000 - $8,999', '$9,000 - $9,999', '$10,000 - 10,999',
       '$11,000 - 11,999', '$12,000 - $14,999', '$15,000 & Over', 'tree cover',
 

In [7]:
columns_to_drop = ['land_use_label', 'subzone', 'planning_area', 'region',
       'temp_2024_04_07_min', 'temp_2024_04_07_max',
       'temp_2024_04_07_median', 'temp_2024_04_08_min', 'temp_2024_04_08_max',
       'temp_2024_04_08_median', 'temp_2024_04_09_min', 'temp_2024_04_09_max',
       'temp_2024_04_09_median', 'temp_2024_04_10_min', 'temp_2024_04_10_max',
       'temp_2024_04_10_median']

X_train = X_train.drop(columns=columns_to_drop)
X_test = X_test.drop(columns=columns_to_drop)

In [8]:
# Remove rows where min_ndvi values is -
X_train = X_train[X_train['min_ndvi'] != '-']
X_test = X_test[X_test['min_ndvi'] != '-']

In [9]:
# Split X and y
y_train = X_train['avg_temp']
X_train = X_train.drop(columns=['avg_temp'])

y_test = X_test['avg_temp']
X_test = X_test.drop(columns=['avg_temp'])

## Model Training

In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 500, step=50)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5)

    regressor = GradientBoostingRegressor(n_estimators=n_estimators,
                                     max_depth=max_depth,
                                     learning_rate=learning_rate,
                                     min_samples_split=min_samples_split,
                                     min_samples_leaf=min_samples_leaf,
                                     random_state=42)
    regressor.fit(X_train_scaled, y_train)

    # Predict and calculate the R2 score
    y_pred = regressor.predict(X_test_scaled)
    score = r2_score(y_test, y_pred)
    return score

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

print("Best trial:")
trial = study.best_trial
print(f"  R2 score: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-04-19 13:25:58,714] A new study created in memory with name: no-name-8ca5a8e5-c4ae-48e1-abb4-22a92a2fe389
[I 2024-04-19 13:25:59,291] Trial 0 finished with value: 0.5028194340904202 and parameters: {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.06686679022037954, 'min_samples_split': 7, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.5028194340904202.
[I 2024-04-19 13:26:00,290] Trial 1 finished with value: 0.42884710353857014 and parameters: {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.022399596476426725, 'min_samples_split': 10, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.5028194340904202.
[I 2024-04-19 13:26:01,982] Trial 2 finished with value: 0.24614518981744427 and parameters: {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.25019182778490673, 'min_samples_split': 6, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.5028194340904202.
[I 2024-04-19 13:26:05,430] Trial 3 finished with value: 0.3376213244934926 and parameters

Best trial:
  R2 score: 0.5292691014867752
  Params: 
    n_estimators: 100
    max_depth: 3
    learning_rate: 0.07598329220207736
    min_samples_split: 9
    min_samples_leaf: 5


In [12]:
regressor = GradientBoostingRegressor(n_estimators=100,
                                     max_depth=3,
                                     learning_rate=0.07598329220207736,
                                     min_samples_split=9,
                                     min_samples_leaf=5,
                                     random_state=42)

regressor.fit(X_train_scaled, y_train)

y_pred = regressor.predict(X_test_scaled)
score = r2_score(y_test, y_pred)

# Calculate the RMSE
rmse = np.sqrt(np.mean((y_test - y_pred)**2))
print(f"RMSE: {rmse}")

# Calculate the R2
r2 = r2_score(y_test, y_pred)
print(f"R2: {r2}")

RMSE: 0.7816571746001129
R2: 0.5292691014867752


In [15]:
# Scale the features (important for SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 500, step=50)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5)

    regressor = GradientBoostingRegressor(n_estimators=n_estimators,
                                     max_depth=max_depth,
                                     learning_rate=learning_rate,
                                     min_samples_split=min_samples_split,
                                     min_samples_leaf=min_samples_leaf,
                                     random_state=42)
    regressor.fit(X_train_scaled, y_train)

    # Predict and calculate the rmse score
    y_pred = regressor.predict(X_test_scaled)
    score = np.sqrt(np.mean((y_test - y_pred)**2))
    return score

# Create a study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)

print("Best trial:")
trial = study.best_trial
print(f"  Rmse score: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2024-04-19 13:59:47,700] A new study created in memory with name: no-name-4dd6a5b1-483e-4f60-9492-1e06ed330079


[I 2024-04-19 13:59:50,122] Trial 0 finished with value: 0.9373264915446476 and parameters: {'n_estimators': 450, 'max_depth': 8, 'learning_rate': 0.12727953124722993, 'min_samples_split': 8, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.9373264915446476.
[I 2024-04-19 13:59:51,503] Trial 1 finished with value: 0.9586094434959125 and parameters: {'n_estimators': 400, 'max_depth': 4, 'learning_rate': 0.285427572078015, 'min_samples_split': 5, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.9373264915446476.
[I 2024-04-19 13:59:53,682] Trial 2 finished with value: 0.9634384942862237 and parameters: {'n_estimators': 400, 'max_depth': 8, 'learning_rate': 0.17151527961943858, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.9373264915446476.
[I 2024-04-19 13:59:54,501] Trial 3 finished with value: 0.9509193651216145 and parameters: {'n_estimators': 150, 'max_depth': 8, 'learning_rate': 0.1815935927091614, 'min_samples_split': 8, 'min_samples_leaf':

Best trial:
  Rmse score: 0.7815773877288932
  Params: 
    n_estimators: 100
    max_depth: 3
    learning_rate: 0.057849013976094876
    min_samples_split: 7
    min_samples_leaf: 4


In [16]:
regressor = GradientBoostingRegressor(n_estimators=100,
                                     max_depth=3,
                                     learning_rate=0.057849013976094876,
                                     min_samples_split=7,
                                     min_samples_leaf=4,
                                     random_state=42)

regressor.fit(X_train_scaled, y_train)

y_pred = regressor.predict(X_test_scaled)
score = r2_score(y_test, y_pred)

# Calculate the RMSE
rmse = np.sqrt(np.mean((y_test - y_pred)**2))
print(f"RMSE: {rmse}")

# Calculate the R2
r2 = r2_score(y_test, y_pred)
print(f"R2: {r2}")

RMSE: 0.7815773877288932
R2: 0.5293651953497733
